In [1]:
 # add additional module import path to import the desired function from the classifier module
# https://stackoverflow.com/questions/34478398/import-local-function-from-a-module-housed-in-another-directory-with-relative-im#answer-35273613
import os
import sys

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd

META_PATH: str = '../data/metacritic'
IMDB_PATH: str = '../data/imdb'

In [3]:
formatted: pd.DataFrame = pd.read_csv(f'{META_PATH}.formatted.csv')
formatted_sub = formatted.iloc[0:32]
formatted

,title,metascore,userscore,summary
0,The Godfather,100,9.2,\n Francis Ford Coppola...
1,Citizen Kane,100,8.4,\n Following the death ...
2,Rear Window,100,8.7,\n A wheelchair-bound p...
3,Casablanca,100,8.8,"\n A Casablanca, Morocc..."
4,Boyhood,100,7.4,\n Filmed over 12 years...


In [7]:
test: pd.DataFrame = pd.read_csv(f'{IMDB_PATH}.test.csv')
test_sub = test.iloc[0:32]
test_sub

,Unnamed: 0,review,sentiment
0,0,"Based on an actual story, John Boorman shows t...",positive
1,1,This is a gem. As a Film Four production - the...,positive
2,2,"I really like this show. It has drama, romance...",positive
3,3,This is the best 3-D experience Disney has at ...,positive
4,4,"Of the Korean movies I've seen, only three had...",positive
5,5,this movie is funny funny funny my favorite qu...,positive
6,6,I'm just starting to explore the so far wonder...,positive
7,7,There is no need for me to repeat the synopsis...,positive
8,8,"I got this movie with my BBC ""Jane Austen Coll...",positive
9,9,"This was a great movie, I would compare it to ...",positive


In [9]:
from classifier._neural import Encoder

testEncoder = Encoder({})

In [10]:
testEncoder.df_encode(formatted_sub, "summary", label="MetaCritic")
formatted_sub

/Users/simon/Documents/04--University/02-Master--Natural-Language-Processing/02--Courses/02--SoSe2022/C - Natural Language Processing/++ Paper ++/++ Code ++/classifier/_neural/encoder.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[self.col_name] = embeds


,metascore,userscore,summary,prediction
title,,,,
The Godfather,1.00,0.92,Francis Ford Coppola's...,"[tensor(-0.4280), tensor(-0.2083), tensor(0.10..."
Citizen Kane,1.00,0.84,Following the death of...,"[tensor(-0.0159), tensor(-0.2666), tensor(0.47..."
Rear Window,1.00,0.87,A wheelchair-bound pho...,"[tensor(-0.3039), tensor(-0.5063), tensor(0.48..."
Casablanca,1.00,0.88,"A Casablanca, Morocco ...","[tensor(-0.2612), tensor(-0.5160), tensor(0.12..."
Boyhood,1.00,0.74,Filmed over 12 years w...,"[tensor(-0.1925), tensor(-0.1002), tensor(0.17..."
Three Colors: Red,1.00,0.85,Krzysztof Kieslowski c...,"[tensor(-0.2618), tensor(-0.2848), tensor(0.39..."
Vertigo,1.00,0.86,Vertigo creates a dizz...,"[tensor(-0.0367), tensor(-0.4942), tensor(0.60..."
Notorious,1.00,0.79,A woman is asked to sp...,"[tensor(-0.1946), tensor(-0.3406), tensor(-0.0..."
Singin' in the Rain,0.99,0.87,A silent film producti...,"[tensor(0.0964), tensor(-0.6862), tensor(-0.12..."


In [11]:
testEncoder.df_encode(test_sub, "review", label="test")
test_sub

/Users/simon/Documents/04--University/02-Master--Natural-Language-Processing/02--Courses/02--SoSe2022/C - Natural Language Processing/++ Paper ++/++ Code ++/classifier/_neural/encoder.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[self.col_name] = embeds


,Unnamed: 0,review,sentiment,prediction
0,0,"Based on an actual story, John Boorman shows t...",positive,"[tensor(-0.0125), tensor(-0.3061), tensor(0.40..."
1,1,This is a gem. As a Film Four production - the...,positive,"[tensor(-0.1354), tensor(-0.5075), tensor(0.36..."
2,2,"I really like this show. It has drama, romance...",positive,"[tensor(0.1687), tensor(-0.2170), tensor(0.522..."
3,3,This is the best 3-D experience Disney has at ...,positive,"[tensor(-0.4735), tensor(-0.7686), tensor(0.52..."
4,4,"Of the Korean movies I've seen, only three had...",positive,"[tensor(-0.2199), tensor(-0.2028), tensor(0.07..."
5,5,this movie is funny funny funny my favorite qu...,positive,"[tensor(-0.0776), tensor(-0.0189), tensor(0.64..."
6,6,I'm just starting to explore the so far wonder...,positive,"[tensor(-0.1093), tensor(-0.1470), tensor(0.50..."
7,7,There is no need for me to repeat the synopsis...,positive,"[tensor(0.1148), tensor(-0.2743), tensor(0.530..."
8,8,"I got this movie with my BBC ""Jane Austen Coll...",positive,"[tensor(-0.4041), tensor(0.0180), tensor(0.160..."
9,9,"This was a great movie, I would compare it to ...",positive,"[tensor(-0.0197), tensor(-0.1570), tensor(0.45..."


In [19]:
import torch
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=False, verbose=int(False))

In [16]:
pool = torch.stack(formatted_sub['prediction'].tolist()).float()

In [31]:
test_sub["metacritic"] = test_sub.parallel_apply(
    lambda row: formatted_sub.iloc[torch.norm(pool - row['prediction'].unsqueeze(0), dim=1).argmin().item()]['metascore'],
    axis=1
)

/var/folders/7n/1kfsdp892135fl2ydt6ttrw00000gn/T/ipykernel_10241/1119956272.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_sub["metacritic"] = test_sub.parallel_apply(


In [32]:
test_sub

,Unnamed: 0,review,sentiment,prediction,metacritic
0,0,"Based on an actual story, John Boorman shows t...",positive,"[tensor(-0.0125), tensor(-0.3061), tensor(0.40...",0.97
1,1,This is a gem. As a Film Four production - the...,positive,"[tensor(-0.1354), tensor(-0.5075), tensor(0.36...",0.99
2,2,"I really like this show. It has drama, romance...",positive,"[tensor(0.1687), tensor(-0.2170), tensor(0.522...",0.97
3,3,This is the best 3-D experience Disney has at ...,positive,"[tensor(-0.4735), tensor(-0.7686), tensor(0.52...",1.00
4,4,"Of the Korean movies I've seen, only three had...",positive,"[tensor(-0.2199), tensor(-0.2028), tensor(0.07...",0.97
5,5,this movie is funny funny funny my favorite qu...,positive,"[tensor(-0.0776), tensor(-0.0189), tensor(0.64...",1.00
6,6,I'm just starting to explore the so far wonder...,positive,"[tensor(-0.1093), tensor(-0.1470), tensor(0.50...",0.97
7,7,There is no need for me to repeat the synopsis...,positive,"[tensor(0.1148), tensor(-0.2743), tensor(0.530...",0.99
8,8,"I got this movie with my BBC ""Jane Austen Coll...",positive,"[tensor(-0.4041), tensor(0.0180), tensor(0.160...",1.00
9,9,"This was a great movie, I would compare it to ...",positive,"[tensor(-0.0197), tensor(-0.1570), tensor(0.45...",0.97


In [34]:
test_sub[['sentiment', 'metacritic']]

,sentiment,metacritic
0,positive,0.97
1,positive,0.99
2,positive,0.97
3,positive,1.00
4,positive,0.97
5,positive,1.00
6,positive,0.97
7,positive,0.99
8,positive,1.00
9,positive,0.97
